It's really handy to have all the DICOM info available in a single DataFrame, so let's create that! In this notebook, we'll just create the DICOM DataFrames. To see how to use them to analyze the competition data, see [this followup notebook](https://www.kaggle.com/jhoward/some-dicom-gotchas-to-be-aware-of-fastai).

First, we'll install the latest versions of pytorch and fastai v2 (not officially released yet) so we can use the fastai medical imaging module.

In [1]:
#default_exp metadata

In [2]:
#export
from rsna_retro.imports import *

Let's take a look at what files we have in the dataset.

In [3]:
#export
set_num_threads(1)
path = Path('~/data/rsna').expanduser()
path_meta = path/'meta'

In [4]:
#export
dir_trn = 'stage_2_train'
dir_tst = 'stage_2_test'
fth_lbl = path_meta/'labels2.fth'
fth_trn = path_meta/'df_trn2.fth'
fth_tst = path_meta/'df_tst2.fth'
fth_trn_comb = path_meta/'df_trn2_comb.fth'

In [5]:
#export
htypes = ['any','epidural','intraparenchymal','intraventricular','subarachnoid','subdural']

In [6]:
# Stage 1 training
# dir_trn = 'stage_1_train_images'
# dir_tst = 'stage_1_test_images'
# fth_lbl = path_meta/'labels.fth'
# fth_trn = path_meta/'df_trn.fth'
# fth_tst = path_meta/'df_tst.fth'

Most lists in fastai v2, including that returned by `Path.ls`, are returned as a [fastai.core.L](http://dev.fast.ai/core.html#L), which has lots of handy methods, such as `attrgot` used here to grab file names.

In [7]:
#export
path_trn = path/dir_trn
fns_trn = path_trn.ls()

path_tst = path/dir_tst
fns_tst = path_tst.ls()

In [8]:
len(fns_trn),len(fns_tst)

(752803, 121232)

We can grab a file and take a look inside using the `dcmread` method that fastai v2 adds.

# Labels

Before we pull the metadata out of the DIMCOM files, let's process the labels into a convenient format and save it for later. We'll use *feather* format because it's lightning fast!

In [9]:
#export
def save_lbls():
    path_lbls = path/f'{dir_trn}.csv'
    if fth_lbl.exists(): return
    lbls = pd.read_csv(path_lbls)
    lbls[["ID","htype"]] = lbls.ID.str.rsplit("_", n=1, expand=True)
    lbls.drop_duplicates(['ID','htype'], inplace=True)
    pvt = lbls.pivot('ID', 'htype', 'Label')
    pvt.reset_index(inplace=True)    
    pvt.to_feather(fth_lbl)

In [10]:
save_lbls()

In [11]:
df_lbls = pd.read_feather(fth_lbl).set_index('ID')

In [12]:
df_lbls.head(8)

,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
ID,,,,,,
ID_000012eaf,0,0,0,0,0,0
ID_000039fa0,0,0,0,0,0,0
ID_00005679d,0,0,0,0,0,0
ID_00008ce3c,0,0,0,0,0,0
ID_0000950d7,0,0,0,0,0,0
ID_0000aee4b,0,0,0,0,0,0
ID_0000ca2f6,0,0,0,0,0,0
ID_0000f1657,0,0,0,0,0,0


In [13]:
df_lbls.mean()

any                 0.143375
epidural            0.004178
intraparenchymal    0.047978
intraventricular    0.034810
subarachnoid        0.047390
subdural            0.062654
dtype: float64

# DICOM Meta

To turn the DICOM file metadata into a DataFrame we can use the `from_dicoms` function that fastai v2 adds. By passing `px_summ=True` summary statistics of the image pixels (mean/min/max/std) will be added to the DataFrame as well (although it takes much longer if you include this, since the image data has to be uncompressed).

In [14]:
def process_metadata(fns, out_f, n_workers=12):
    if out_f.exists(): return
    df = pd.DataFrame.from_dicoms(fns, px_summ=True, window=dicom_windows.brain, n_workers=12)
    df.to_feather(out_f)
    return df
    

In [15]:
process_metadata(fns_tst, fth_tst)

In [16]:
#export
df_tst = pd.read_feather(fth_tst)

In [17]:
df_tst.head()

,SOPInstanceUID,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,...,PixelSpacing1,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,img_min,img_max,img_mean,img_std,img_pct_window
0,ID_de973ed39,CT,ID_9a848de8,ID_636648ddc3,ID_3334944046,,-125.0,1.0,1,MONOCHROME2,...,0.488281,1.0,47.0,1.0,80.0,0,2766,458.317307,581.025693,0.238255
1,ID_124564e24,CT,ID_0f6dde7d,ID_5f6fc2a2cd,ID_7514a7fb7b,,-139.0,1.0,1,MONOCHROME2,...,0.488281,1.0,40.0,1.0,80.0,0,2441,481.852821,565.443686,0.252171
2,ID_15c8e85ea,CT,ID_00556373,ID_f5ef8bf6ea,ID_41bab3d33d,,-125.0,1.0,1,MONOCHROME2,...,0.488281,NaN,NaN,NaN,NaN,-2000,3299,19.316921,1172.887786,0.165970
3,ID_0e37b7ef5,CT,ID_ba67f475,ID_75e823b787,ID_cdbc8473f3,,-131.0,1.0,1,MONOCHROME2,...,0.488281,1.0,36.0,1.0,80.0,6,2566,416.983677,571.039439,0.232601
4,ID_25fab03f0,CT,ID_ebec6b48,ID_bb3711c18b,ID_7bbbbdb05f,,-119.5,1.0,1,MONOCHROME2,...,0.466797,1.0,36.0,1.0,80.0,4,2420,347.349949,566.757696,0.098629


In [18]:
process_metadata(fns_trn, fth_trn)

In [19]:
df_trn = pd.read_feather(fth_trn)

In [20]:
df_trn.head()

,SOPInstanceUID,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,...,PixelSpacing1,img_min,img_max,img_mean,img_std,img_pct_window,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1
0,ID_352e89f1c,CT,ID_d557ddd2,ID_05074a0d95,ID_be6165332c,,-125.000000,1.0,1,MONOCHROME2,...,0.488281,-2000,2787,35.112926,1166.720843,0.164139,NaN,NaN,NaN,NaN
1,ID_3cf4fb50f,CT,ID_16b2ad86,ID_c3a404ea2e,ID_2c1454e208,,-125.000000,1.0,1,MONOCHROME2,...,0.488281,0,2412,234.549896,392.132243,0.076015,1.0,36.0,1.0,80.0
2,ID_e3674b189,CT,ID_eb712bf0,ID_db83193795,ID_e1facea145,,-125.000000,1.0,1,MONOCHROME2,...,0.488281,-2000,2749,50.591320,1216.541625,0.243259,NaN,NaN,NaN,NaN
3,ID_2a8702d25,CT,ID_ff137633,ID_d17053848c,ID_7098f7c836,,-126.437378,1.0,1,MONOCHROME2,...,0.494863,0,2806,482.248981,571.235614,0.241489,NaN,NaN,NaN,NaN
4,ID_7be0f1b3c,CT,ID_cd9169c2,ID_b42de79024,ID_f5bd86b25b,,-125.000000,1.0,1,MONOCHROME2,...,0.488281,-2000,2776,10.762859,1164.588862,0.251751,NaN,NaN,NaN,NaN


In [21]:
if not fth_trn_comb.exists():
    df_comb = df_trn.join(df_lbls, 'SOPInstanceUID')
    df_comb.to_feather(fth_trn_comb)

In [22]:
#export
df_comb = pd.read_feather(fth_trn_comb)

In [23]:
df_comb.head()

,SOPInstanceUID,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
0,ID_352e89f1c,CT,ID_d557ddd2,ID_05074a0d95,ID_be6165332c,,-125.000000,1.0,1,MONOCHROME2,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
1,ID_3cf4fb50f,CT,ID_16b2ad86,ID_c3a404ea2e,ID_2c1454e208,,-125.000000,1.0,1,MONOCHROME2,...,1.0,36.0,1.0,80.0,0,0,0,0,0,0
2,ID_e3674b189,CT,ID_eb712bf0,ID_db83193795,ID_e1facea145,,-125.000000,1.0,1,MONOCHROME2,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
3,ID_2a8702d25,CT,ID_ff137633,ID_d17053848c,ID_7098f7c836,,-126.437378,1.0,1,MONOCHROME2,...,NaN,NaN,NaN,NaN,1,0,1,1,0,0
4,ID_7be0f1b3c,CT,ID_cd9169c2,ID_b42de79024,ID_f5bd86b25b,,-125.000000,1.0,1,MONOCHROME2,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


## Split Patients by IDS

In [24]:
# sops = set(Path('val_sops.pkl').load())

In [25]:
#export
def patient_cv(idx, patient_grps): return np.concatenate([patient_grps[o] for o in range_of(patient_grps) if o!=idx])

def split_data(df, cv_idx, patient_grps):
    idx = L.range(df)
    pgrp = patient_cv(cv_idx, patient_grps)
    mask = df.PatientID.isin(pgrp)
    return idx[mask],idx[~mask]

In [26]:
#export
nfold = 8
ifold = 0
set_seed(42)
patients = df_comb.PatientID.unique()
np.random.shuffle(patients)
patient_grps = np.array_split(patients, nfold)
splits = split_data(df_comb, ifold, patient_grps)

In [27]:

# fns = L(list(df_comb.index))
# patients = df_comb.PatientID.unique()
# np.random.shuffle(patients)
# patient_grps = Path('patient_grps.pkl').load()
# val_sops = Path('val_sops.pkl').load()
# idx = L.range(df_comb)
# mask = df_comb.index.isin(set(val_sops))
# splits = idx[~mask],idx[mask]

## Export

In [28]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 01_data_01_metadata_stage2.ipynb.
Converted 01_data_02_preprocess_windows.ipynb.
Converted 02_train_01_train.ipynb.
Converted 04_orig_replace_ashaw_refactor.ipynb.
Converted 04_replace_ashaw_refactor.ipynb.
Converted 04b_orig_replace_ashaw_refactor.ipynb.
Converted 10_qure.ipynb.
Converted 12_merge.ipynb.
Converted 14_xgboost.ipynb.
Converted 16_slice_e2e-shallow.ipynb.
Converted 16b_orig_slice_e2e-shallow.ipynb.
This cell doesn't have an export destination and was ignored:
e
Converted 17_slice_model-deep.ipynb.
Converted 21_cleanup-nocrop2.ipynb.
Converted 26_submit_final.ipynb.
Converted 27_ensemble_tabular_nn.ipynb.
Converted 99_index.ipynb.
Converted cleanup-combine-qure.ipynb.
Converted delete_03b_cleanup-tif.ipynb.
Converted submit.ipynb.
Converted walkthru.ipynb.
Converted x00_tcia-ct-segm-prep.ipynb.
Converted x00_tcia-ct-segm-train.ipynb.
